# Liner regressiya orqali GYM atletlarini kaloriya sarfini aniqlash

In [10]:
#Kerakli kutub xonalarni o'rnatib olamiz

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score



In [11]:
# Dataset ni dasturga o'rnatib olamiz
# Muvofiqiyatli o'rnatilgan datasetning ma'lum qismini ko'ramiz

data_f = pd.read_csv('gym_members_exercise_tracking.csv')
data_f.head()

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,Calories_Burned
0,56,Male,88.3,1.71,180.0,157.0,60.0,1.69,Yoga,12.6,3.5,4.0,3.0,1313
1,46,Female,74.9,1.53,179.0,151.0,66.0,1.30,HIIT,33.9,2.1,4.0,2.0,883
2,32,Female,68.1,1.66,167.0,122.0,54.0,1.11,Cardio,33.4,2.3,4.0,2.0,677
3,25,Male,53.2,1.70,190.0,164.0,56.0,0.59,Strength,28.8,2.1,3.0,1.0,532
4,38,Male,46.1,1.79,188.0,158.0,68.0,0.64,Strength,29.2,2.8,3.0,1.0,556


In [12]:
# dataset haqida ma'lumot olamiz

data_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 973 entries, 0 to 972
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Age                            973 non-null    int64  
 1   Gender                         973 non-null    object 
 2   Weight (kg)                    973 non-null    float64
 3   Height (m)                     973 non-null    float64
 4   Max_BPM                        972 non-null    float64
 5   Avg_BPM                        972 non-null    float64
 6   Resting_BPM                    970 non-null    float64
 7   Session_Duration (hours)       971 non-null    float64
 8   Workout_Type                   970 non-null    object 
 9   Fat_Percentage                 967 non-null    float64
 10  Water_Intake (liters)          968 non-null    float64
 11  Workout_Frequency (days/week)  972 non-null    float64
 12  Experience_Level               972 non-null    flo

In [13]:
# dataset da null value larni tekshirib olamiz

data_f.isnull().sum()

Age                              0
Gender                           0
Weight (kg)                      0
Height (m)                       0
Max_BPM                          1
Avg_BPM                          1
Resting_BPM                      3
Session_Duration (hours)         2
Workout_Type                     3
Fat_Percentage                   6
Water_Intake (liters)            5
Workout_Frequency (days/week)    1
Experience_Level                 1
Calories_Burned                  0
dtype: int64

In [14]:
#Tushib qolgan qiymatlari bor ustunlarni o'zini alohida aniqlash

missing_columns=data_f.isnull().sum()
missing_columns=missing_columns[missing_columns>0]
if not missing_columns.empty:
    print('Tushib qolgan elementlari bor ustunlar:')
    print(missing_columns)
else:
    print('Bundaty ustunlar mavjud emas')







Tushib qolgan elementlari bor ustunlar:
Max_BPM                          1
Avg_BPM                          1
Resting_BPM                      3
Session_Duration (hours)         2
Workout_Type                     3
Fat_Percentage                   6
Water_Intake (liters)            5
Workout_Frequency (days/week)    1
Experience_Level                 1
dtype: int64


In [ ]:
# Bir xil raqamli qiymatga o'tib olamiz. Bunda Label encoderdan foydalanamiz
label_encoder = LabelEncoder()

for column in data_f.columns:
    if data_f[column].isnull().sum()>0:
        if data_f[column].dtype=='object':
            data_f[column].fillna(data_f[column].mode()[0], inplace=True)
            data_f[column] = label_encoder.fit_transform(data_f[column])
        else:
            data_f[column].fillna(data_f[column].mean(), inplace=True)

data_f.info()

# data_f['Gender'] = label_encoder.fit_transform(data_f['Gender'])
# data_f['Workout_Type'] = label_encoder.fit_transform(data_f['Workout_Type'])



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 973 entries, 0 to 972
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Age                            973 non-null    int64  
 1   Gender                         973 non-null    int64  
 2   Weight (kg)                    973 non-null    float64
 3   Height (m)                     973 non-null    float64
 4   Max_BPM                        973 non-null    float64
 5   Avg_BPM                        973 non-null    float64
 6   Resting_BPM                    973 non-null    float64
 7   Session_Duration (hours)       973 non-null    float64
 8   Workout_Type                   973 non-null    int64  
 9   Fat_Percentage                 973 non-null    float64
 10  Water_Intake (liters)          973 non-null    float64
 11  Workout_Frequency (days/week)  973 non-null    float64
 12  Experience_Level               973 non-null    flo

In [41]:
# dataset tozalandandan keyin uni input va outputga bo'lib olamiz

x = data_f.drop('Calories_Burned', axis=1)
y = data_f['Calories_Burned']

In [42]:
# datasetimizni train, test va baxolash uchun 0.7:0.15:0.15 bo'lib olamiz

x_train,x_temp,y_train, y_temp=train_test_split(x,y,test_size=0.3, random_state=42)
x_val,x_test,y_val, y_test=train_test_split(x_temp,y_temp,test_size=0.5,random_state=42)


In [43]:
len(x_test)

146

In [44]:
# model train qilamiz

model=LinearRegression()

model.fit(x_train,y_train)


LinearRegression()

In [45]:
#Predicting
y_pred=model.predict(x_test)

y_pred

array([1104.765993  , 1204.14339244,  960.33695066,  419.34490245,
        434.7928491 ,  815.77659176,  490.71747542, 1134.65338682,
       1196.48747787, 1140.01580608, 1324.69048755, 1247.23967788,
       1159.43533021,  843.484923  ,  399.85075853,  482.65374611,
        750.32889234, 1305.81031127,  877.05039316,  636.90305775,
       1322.68582375, 1182.16845667,  760.41812716, 1219.41390253,
        770.33426998, 1032.66860032, 1303.93667655, 1285.81874902,
        508.87724706,  589.2755341 ,  309.22379675,  701.1232462 ,
        579.63288463,  820.59155498,  710.1414754 ,  878.57229426,
        936.72884534,  608.93096144, 1384.14753282, 1240.01710905,
        377.2440715 ,  618.34689629,  945.55210466, 1180.3742281 ,
        921.24448647,  785.82844021,  819.90780951,  474.39366829,
        842.96175316,  330.54307822,  942.12672194, 1315.46760874,
       1116.10291382,  707.34355503,  519.15431541, 1034.53824722,
       1204.19315948,  842.12906246,  958.49434666,  820.80261

In [46]:
# model qanchalik aniq ishlashini tekshiramiz
from sklearn.metrics import mean_squared_error, r2_score

mse=mean_squared_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)

print(mse)
print(r2)

1672.0090060801058
0.9774334612309732


In [47]:
scaler_x=StandardScaler()
scaler_y=StandardScaler()

x_train_scaled=scaler_x.fit_transform(x_train)
x_test_scaled=scaler_x.fit_transform(x_test)
y_train_scaled=scaler_y.fit_transform(y_train.values.reshape(-1,1))
y_test_scaled=scaler_y.fit_transform(y_test.values.reshape(-1,1))

In [48]:
model=LinearRegression()
model.fit(x_train_scaled,y_train_scaled)


LinearRegression()

In [49]:
y_pred_scaled=model.predict(x_test_scaled)

In [50]:
mse=mean_squared_error(y_test_scaled,y_pred_scaled)
r2=r2_score(y_test_scaled,y_pred_scaled)

print(mse)
print(r2)

0.020785911949086668
0.9792140880509134
